# Antes de empezar:
- Lee el archivo README.md
- Comenta tanto como puedas y utiliza los recursos (archivo README.md)
- ¡Feliz aprendizaje!

In [5]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest
from scipy.stats import ttest_ind, norm, t
from scipy.stats import f_oneway
from scipy.stats import sem

# Reto 1 - Explorando los Datos

En este reto, examinaremos todos los salarios de los empleados de la Ciudad de Chicago. Comenzaremos cargando el conjunto de datos y examinando su contenido.

In [7]:
# Run this code:
salaries = pd.read_csv('../data/Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examina el conjunto de datos `salaries` utilizando la función `head` que se muestra a continuación.

In [8]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


# Reto 2 - Pruebas de Hipótesis

En esta sección del laboratorio, vamos a probar si el salario por hora de todos los trabajadores por hora es significativamente diferente de $30/hora. Importa la función correcta para la prueba de una muestra desde scipy y realiza la prueba de hipótesis para un intervalo de confianza de dos lados del 95%.

In [13]:

# Cargar el conjunto de datos
salaries = pd.read_csv('../data/Current_Employee_Names__Salaries__and_Position_Titles.csv')

# Filtrar los datos para incluir solo trabajadores por hora
hourly_workers = salaries[salaries['Salary or Hourly'] == 'Hourly']
hourly_rates = hourly_workers['Hourly Rate'].dropna()

# Parámetros de la muestra
sample_mean = np.mean(hourly_rates)
sample_std = np.std(hourly_rates, ddof=1)
n = len(hourly_rates)
mu = 30

# Cálculo del estadístico t
t_statistic = (sample_mean - mu) / (sample_std / np.sqrt(n))

# Grados de libertad
df = n - 1

# Cálculo del valor p
p_value = 2 * (1 - t.cdf(abs(t_statistic), df))

# Mostrar resultados
print("Estadístico t:", t_statistic)
print("Valor p:", p_value)

# Interpretación de los resultados
if p_value < 0.05:
    print("Rechazamos la hipótesis nula: el salario por hora promedio es significativamente diferente de $30.")
else:
    print("No rechazamos la hipótesis nula: el salario por hora promedio no es significativamente diferente de $30.")


Estadístico t: 20.619805785501907
Valor p: 0.0
Rechazamos la hipótesis nula: el salario por hora promedio es significativamente diferente de $30.


# Reto 3 - Construyendo Intervalos de Confianza

Aunque probar nuestra hipótesis es una excelente manera de obtener evidencia empírica para aceptar o rechazar la hipótesis, otra forma de recopilar evidencia es creando un intervalo de confianza. Un intervalo de confianza nos brinda información sobre la media verdadera de la población. Por lo tanto, para un intervalo de confianza del 95%, estamos seguros en un 95% de que la media de la población se encuentra dentro del intervalo de confianza.

Para leer más sobre los intervalos de confianza, haz clic [aquí](https://es.wikipedia.org/wiki/Intervalo_de_confianza).

En la celda a continuación, construiremos un intervalo de confianza del 95% para el salario por hora medio de todos los trabajadores por hora.

El intervalo de confianza se calcula en SciPy utilizando la función `t.interval`. Puedes leer más sobre esta función [aquí](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

Para calcular el intervalo de confianza del salario por hora, usa 0.95 para el nivel de confianza, el número de filas - 1 para los grados de libertad, la media de la muestra para el parámetro de ubicación y el error estándar para la escala. El error estándar se puede calcular utilizando [esta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) función en SciPy.

In [11]:

# Cargar el conjunto de datos
salaries = pd.read_csv('../data/Current_Employee_Names__Salaries__and_Position_Titles.csv')

# Filtrar los datos para incluir solo trabajadores por hora
hourly_workers = salaries[salaries['Salary or Hourly'] == 'Hourly']

# Extraer las tarifas por hora y eliminar valores nulos
hourly_rates = hourly_workers['Hourly Rate'].dropna()

# Calcular la media y el error estándar de los salarios por hora
mean_hourly_rate = hourly_rates.mean()
std_error = sem(hourly_rates)

# Calcular el intervalo de confianza del 95%
confidence_interval = t.interval(
    0.95,  # Nivel de confianza
    len(hourly_rates) - 1,  # Grados de libertad
    loc=mean_hourly_rate,  # Media de la muestra
    scale=std_error  # Error estándar
)

# Mostrar el intervalo de confianza
print("Intervalo de confianza del 95% para el salario por hora medio:", confidence_interval)


Intervalo de confianza del 95% para el salario por hora medio: (32.52345834488529, 33.05365708767727)


# Reto 4 - Pruebas de Hipótesis de Proporciones

Otro tipo de prueba de una muestra es una prueba de hipótesis de proporciones. En esta prueba, examinamos si la proporción de un grupo en nuestra muestra es significativamente diferente de una fracción.

Puedes leer más sobre las pruebas de proporción de una muestra [aquí](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

En la celda a continuación, utiliza la función `proportions_ztest` de `statsmodels` para realizar una prueba de hipótesis que determinará si el número de trabajadores por hora en la Ciudad de Chicago es significativamente diferente del 25% en el nivel de confianza del 95%.

In [14]:

from statsmodels.stats.proportion import proportions_ztest

# Cargar el conjunto de datos
salaries = pd.read_csv('../data/Current_Employee_Names__Salaries__and_Position_Titles.csv')

# Contar el número de trabajadores por hora
hourly_count = salaries[salaries['Salary or Hourly'] == 'Hourly'].shape[0]

# Total de empleados
total_count = salaries.shape[0]

# Proporción esperada
p0 = 0.25

# Realizar la prueba z para proporciones
z_stat, p_value = proportions_ztest(count=hourly_count, nobs=total_count, value=p0, alternative='two-sided')

# Mostrar resultados
print("Estadístico z:", z_stat)
print("Valor p:", p_value)

# Interpretación de los resultados
if p_value < 0.05:
    print("Rechazamos la hipótesis nula: la proporción de trabajadores por hora es significativamente diferente del 25%.")
else:
    print("No rechazamos la hipótesis nula: la proporción de trabajadores por hora no es significativamente diferente del 25%.")

Estadístico z: -3.5099964213703005
Valor p: 0.0004481127249057967
Rechazamos la hipótesis nula: la proporción de trabajadores por hora es significativamente diferente del 25%.
